# Creating aggregate bike data

In [1]:
import pandas as pd
from datetime import datetime, date
import os

In [2]:
# Looks at monthly bike data and selects relevant columns

def get_columns(df):
    columns = ['Departure','Covered distance (m)','Duration (sec.)']
    for c in df:
        if c.startswith(('Electric','Departure temperature','Return temperature')):
            columns += [c]
    return columns

In [3]:
# Associates an operation to perform on each column

def operations(cols):
    ops = {'Departure':'count',
           'Covered distance (m)':'sum',
           'Duration (sec.)':'sum'}
    for c in cols:
        if c.startswith('Electric'):
            ops[c] = 'sum'
        if c.startswith(('Departure temperature','Return temperature')):
            ops[c] = 'mean'
    return ops

In [4]:
# Specifies new columnn names to be used after
# operations have been applied

def rename(cols):
    names = {'Departure':'Bike trips',
             'Covered distance (m)':'Total distance (m)',
             'Duration (sec.)':'Total duration (sec)',}
    for c in cols:
        if c.startswith('Electric'):
            names[c] = 'Electric bike trips'
        if c.startswith('Departure temperature'):
            names[c] = 'Mean departure temperature (C)'
        if c.startswith('Return temperature'):
            names[c] = 'Mean return temperature (C)'
    return names

In [5]:
def get_date(d):
    return d.date()

In [6]:
aggregate_data = pd.DataFrame({})

# names of monthly data files
files = [file_name for file_name in os.listdir('raw data') if file_name.endswith('.csv')]

for file_name in files:
    # Load a monthly data file
    raw_data = pd.read_csv('raw data/'+file_name, parse_dates=['Departure'])
    print(file_name)

    # Identify relevant columns
    cols = get_columns(raw_data)
    proc_data = raw_data[cols].copy().dropna()

    # Group data by date (in 'Departure' column)
    # and apply an operation (e.g. sum, mean) to each column
    proc_data = proc_data.groupby(proc_data['Departure'].apply(get_date)).agg(operations(cols))
    proc_data.index.name = 'Date'

    # Rename columns after grouping and transforming
    proc_data.rename(columns=rename(cols), inplace=True)

    # Add the processed monthly data to the aggregate DataFrame
    aggregate_data = pd.concat([aggregate_data, proc_data], ignore_index = False)

aggregate_data.sort_index(ascending=True, inplace=True)

September 2019.csv
September 2018.csv
September 2023.csv
September 2022.csv
September 2020.csv
September 2021.csv
December 2022.csv


/var/folders/87/tjh6zp2562q1vd9_tgdvt1500000gn/T/ipykernel_20049/1840006905.py:8: DtypeWarning: Columns (1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('raw data/'+file_name, parse_dates=['Departure'])


February 2019.csv
February 2024.csv
February 2018.csv
December 2023.csv
December 2021.csv
Novemeber 2021.csv
December 2020.csv
December 2018.csv


/var/folders/87/tjh6zp2562q1vd9_tgdvt1500000gn/T/ipykernel_20049/1840006905.py:8: DtypeWarning: Columns (1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('raw data/'+file_name, parse_dates=['Departure'])


February 2023.csv
February 2022.csv
December 2019.csv
February 2020.csv
February 2021.csv
May 2023.csv
August 2020.csv
January 2018.csv
April 2019.csv
January 2024.csv
October 2019.csv
October 2018.csv
ALL of 2017.csv
April 2018.csv
January 2019.csv
April 2024.csv
August 2021.csv
May 2022.csv
March 2019.csv
May 2020.csv
August 2023.csv
August 2022.csv
May 2021.csv
March 2024.csv
March 2018.csv
March 2020.csv
May 2019.csv
January 2022.csv
April 2023.csv


...

October 2023.csv
October 2022.csv
April 2022.csv
January 2023.csv
May 2018.csv


/var/folders/87/tjh6zp2562q1vd9_tgdvt1500000gn/T/ipykernel_20049/1840006905.py:8: DtypeWarning: Columns (1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('raw data/'+file_name, parse_dates=['Departure'])


March 2021.csv
March 2023.csv


/var/folders/87/tjh6zp2562q1vd9_tgdvt1500000gn/T/ipykernel_20049/1840006905.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('raw data/'+file_name, parse_dates=['Departure'])


August 2019.csv
January 2021.csv
April 2020.csv
October 2020.csv
October 2021.csv
April 2021.csv
January 2020.csv
August 2018.csv
March 2022.csv
July 2020.csv
June 2020.csv
November 2023.csv
November 2022.csv
June 2021.csv
July 2021.csv
July 2023.csv
June 2023.csv
November 2020.csv
June 2022.csv
July 2022.csv
November 2019.csv
November 2018.csv
July 2019.csv
June 2019.csv
June 2018.csv
July 2018.csv


We now have a DataFrame containing all bike data compressed to a daily scale.  Note that the first day of each month appears in the raw data file for that month *and* the preceding one.  (E.g.  There are April 1, 2024 bike trips logged in both the April 2024 and March 2024 files.) So our aggregate DataFrame will have a repeated index at the first of each month.  For example:

In [7]:
aggregate_data.loc[date(2024,4,1)]

,Bike trips,Total distance (m),Total duration (sec),Mean departure temperature (C),Mean return temperature (C),Electric bike trips
Date,,,,,,
2024-04-01,12,16334.0,32841.0,10.833333,10.250000,0
2024-04-01,3628,11082297.0,4953672.0,16.729327,17.100331,1256


We can combine these rows by summing the `Bike trips`, `Total distance (m)`, `Total duration`, and `Electric bike trips` columns and taking a weighted average in the `Mean departure temperature (C)` and `Mean return temperature (C)` columns.

In [8]:
# Switch to integer indexing
aggregate_data.reset_index(inplace=True)

# Get locations of repeated dates
bad_indices = [i for i in range(len(aggregate_data.index)-1)
               if aggregate_data.loc[i,'Date'] == aggregate_data.loc[i+1,'Date']]

for i in bad_indices:
    # Get data from both rows
    trips1, trips2 = aggregate_data.loc[[i,i+1],'Bike trips']
    dist1, dist2 = aggregate_data.loc[[i,i+1],'Total distance (m)']
    dur1, dur2 = aggregate_data.loc[[i,i+1],'Total duration (sec)']
    dep_temp1, dep_temp2 = aggregate_data.loc[[i,i+1],'Mean departure temperature (C)']
    ret_temp1, ret_temp2 = aggregate_data.loc[[i,i+1],'Mean return temperature (C)']
    elec1, elec2 = aggregate_data.loc[[i,i+1],'Electric bike trips']

    # Record new combined data in the first row
    aggregate_data.loc[i,'Bike trips'] = trips1+trips2
    aggregate_data.loc[i,'Total distance (m)'] = dist1+dist2
    aggregate_data.loc[i,'Total duration (sec)'] = dur1+dur2
    aggregate_data.loc[i,'Mean departure temperature (C)'] = (trips1*dep_temp1 + trips2*dep_temp2)/(trips1+trips2)
    aggregate_data.loc[i,'Mean return temperature (C)'] = (trips1*ret_temp1 + trips2*ret_temp2)/(trips1+trips2)
    aggregate_data.loc[i,'Electric bike trips'] = elec1+elec2

# Drop all of the 'second' rows
aggregate_data.drop([i+1 for i in bad_indices],inplace=True)

# Switch back to date indexing
aggregate_data.set_index('Date', inplace=True)

Now we have:

In [9]:
aggregate_data.loc[date(2024,4,1)]

Bike trips                              3640
Total distance (m)                11098631.0
Total duration (sec)               4986513.0
Mean departure temperature (C)      16.70989
Mean return temperature (C)        17.077747
Electric bike trips                     1256
Name: 2024-04-01, dtype: object

In [10]:
any(aggregate_data.index.duplicated())

False

In [11]:
# aggregate_data.to_csv('aggregate data.csv')